# Cellular internalization

##### Load the CSV file into a DataFrame and reading the data

In [ ]:
import pandas as pd

df = pd.read_csv("Cellular internalization.csv")
df

##### Data analysis (Fe)

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

dataFe = {
    'Nanoparticle concentration(µg/ml)': df['Nanoparticle concentration(µg/ml)'],
    'A_Fe': df.iloc[0, 1:4].values,
    'B_Fe': df.iloc[1, 1:4].values,
    'C_Fe': df.iloc[2, 1:4].values,
    'D_Fe': df.iloc[3, 1:4].values
}

rows = []
for conc, values in zip(dataFe['Nanoparticle concentration(µg/ml)'], zip(dataFe['A_Fe'], dataFe['B_Fe'], dataFe['C_Fe'], dataFe['D_Fe'])):
    for group, value in zip(['A_Fe', 'B_Fe', 'C_Fe', 'D_Fe'], values):
        rows.append({
            'Nanoparticle concentration(µg/ml)': conc,
            'Group': group,
            'Values': value
        })

df_Fe = pd.DataFrame(rows)

# One-way ANOVA
model = ols('Values ~ C(Group)', data=df_Fe).fit()

anova_table = sm.stats.anova_lm(model, typ=1)
print("ANOVA Table:")
print(anova_table)

# Tukey's test
tukey = pairwise_tukeyhsd(endog=df_Fe['Values'], groups=df_Fe['Group'], alpha=0.05)
print("\nTukey's HSD Test:")
print(tukey)

##### Data analysis (Au)

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Extract the data
dataAu = {
    'Nanoparticle concentration(µg/ml)': df['Nanoparticle concentration(µg/ml)'],
    'A_Au': df.iloc[0, 4:7].values,
    'B_Au': df.iloc[1, 4:7].values,
    'C_Au': df.iloc[2, 4:7].values,
    'D_Au': df.iloc[3, 4:7].values
}

rows = []
for conc, values in zip(dataAu['Nanoparticle concentration(µg/ml)'], zip(dataAu['A_Au'], dataAu['B_Au'], dataAu['C_Au'], dataAu['D_Au'])):
    for group, value in zip(['A_Au', 'B_Au', 'C_Au', 'D_Au'], values):
        rows.append({
            'Nanoparticle concentration(µg/ml)': conc,
            'Group': group,
            'Values': value
        })

df_Au = pd.DataFrame(rows)

# One-way ANOVA
model = ols('Values ~ C(Group)', data=df_Au).fit()

anova_table = sm.stats.anova_lm(model, typ=1)
print("ANOVA Table:")
print(anova_table)

# Tukey's test
tukey = pairwise_tukeyhsd(endog=df_Au['Values'], groups=df_Au['Group'], alpha=0.05)
print("\nTukey's HSD Test:")
print(tukey)

##### Plot the results

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
from comparison import pairwise_comparison

df['Mean_FeCon'] = df[['Fe concentration per cell1(pg)', 'Fe concentration per cell2(pg)', 'Fe concentration per cell3(pg)']].mean(axis=1)
df['Mean_AuCon'] = df[['Au concentration per cell1(pg)', 'Au concentration per cell2(pg)', 'Au concentration per cell3(pg)']].mean(axis=1)

df['SEM_Fe'] = df[['Fe concentration per cell1(pg)', 'Fe concentration per cell2(pg)', 'Fe concentration per cell3(pg)']].sem(axis=1)
df['SEM_Au'] = df[['Au concentration per cell1(pg)', 'Au concentration per cell2(pg)', 'Au concentration per cell3(pg)']].sem(axis=1)

plt.figure(figsize=(10, 7))
space_between_bars = 0.175
bar_width = 0.25
x_ticks = np.arange(len(df))

plt.bar(x_ticks - space_between_bars, df['Mean_FeCon'], width=bar_width, color='gray', edgecolor='black', linewidth=3.5, label='Fe')
plt.bar(x_ticks + space_between_bars, df['Mean_AuCon'], width=bar_width, color='yellow', edgecolor='olive', linewidth=3.5, label='Au')

errorbar_Fe = plt.errorbar(x_ticks - space_between_bars, df['Mean_FeCon'], yerr=df['SEM_Fe'], capsize=8, color='black', ecolor='black', markersize=0, fmt='_', elinewidth=3)
errorbar_Au = plt.errorbar(x_ticks + space_between_bars, df['Mean_AuCon'], yerr=df['SEM_Au'], capsize=8, color='olive', ecolor='olive', markersize=0, fmt='_', elinewidth=3)

for cap in errorbar_Fe[1]:
    cap.set_markeredgewidth(2)
    cap.set_path_effects([path_effects.Stroke(linewidth=2), path_effects.Normal()])
for cap in errorbar_Au[1]:
    cap.set_markeredgewidth(2)
    cap.set_path_effects([path_effects.Stroke(linewidth=2), path_effects.Normal()])

font_properties = {'fontweight': 'bold', 'fontsize': 20, 'fontfamily': 'Arial'}
number_font_properties = {'fontweight': 'bold', 'fontsize': 20, 'fontfamily': 'Arial'}
legend_font_properties = {'size': 20, 'family': 'Arial'}

plt.xlabel('Nanoparticle concentration (µg/ml)', **font_properties)
plt.ylabel('Concentration per cell (pg)', **font_properties)
plt.xticks(x_ticks, df['Nanoparticle concentration(µg/ml)'], ha='center', **number_font_properties)
plt.yticks([0, 50, 100, 150], **number_font_properties)
plt.legend()

plt.tick_params(axis='both', which='both', length=15, width=3, labelsize=20)

ax = plt.gca()
ax.spines['left'].set_linewidth(3)
ax.spines['bottom'].set_linewidth(3)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

pairwise_comparison(0, 1, 0.009, x_ticks-0.175, df['Mean_FeCon'].tolist(), yerr=df['SEM_Fe'].tolist(), barh=0.02, fs=18, bold=True, line_width=2)
pairwise_comparison(1, 2, 0.0001, x_ticks-0.175, df['Mean_FeCon'].tolist(), yerr=df['SEM_Fe'].tolist(), barh=0.02, fs=18, bold=True, line_width=2)
pairwise_comparison(2, 3, 0, x_ticks-0.175, df['Mean_FeCon'].tolist(), yerr=df['SEM_Fe'].tolist(), barh=0.02, fs=18, bold=True, line_width=2)

plt.ylim(bottom=0, top=150)
plt.legend(loc='upper left', bbox_to_anchor=(1.1, 1), prop=legend_font_properties, frameon=False)
plt.savefig('Cellular_internalization_graph.tif', dpi=600, format='tif', bbox_inches='tight')
plt.show()